# Dataset Hazırlama (Data Preprocessing)
Bu bölümde dataset nasıl bir değişikliklere uğradı veri nerelerden toplandı gibi konulara değinilecek. Yani kısacası dataset uğradığı bütün değişiklikler anlatılacaktır. Burada değinilen aşamalar gerçekleştirilmesi sonrasında yeni bir dataset elde edildi. Preprocess işlemleri sonrası detaylar *`main.ipynb`* dosyasında ayrıntılı bir şekilde anlatılmıştır. Dataset'in toplanmasında 2 kaynak kullanılmıştır (Kaggle: Youtube Trending Video Dataset, Youtube API).
**NOT:** Bu dosyada bulunan kodlar tek seferlik çalıştırma gereği olduğu için daha önceden çalıştırılıp dataset üzerinde düzenlemeler yapılmıştır. Bu dosyada sadece yapılan işlemler değinilmiştir.

### Kullanılan Kütüphaneler

In [ ]:
import pandas as pd
import requests
import random
import time
import os
from tqdm import tqdm
from datetime import datetime, timedelta

&nbsp;
### Dataset'in Okunması

In [ ]:
FILE_PATH = "dataset.csv"
df = pd.read_csv(FILE_PATH)

&nbsp;
### Sütunların temizlenmesi
Birleştirilen dataset te bazı istenilmeyen sütunlar vardı (dislike ve ratings_disabled) bu veriler 2022 sonrasında artık Youtube API'dan kaldırılmıştır O yüzden bizim kaggle'dan alınan datasetten silinmesi gerekiyordu.


In [ ]:
cols_to_delete = ["ratings_disabled", "dislikes"]
for col in cols_to_delete:
    if col in df.columns:
        df = df.drop(columns=[col])
        print(f"'{col}' sütunu silindi.")
    else:
        print(f"'{col}' sütunu zaten yok.")

&nbsp;
### Sayısal değerlere dönüştürme
True ve False olan bazı sütunlar sayısal değerlere 0,1 olarak dönüştürülmüştür (comments_disabled).

In [ ]:
if "comments_disabled" in df.columns:
    df["comments_disabled"] = df["comments_disabled"].replace({
        True: 1,
        False: 0,
        "True": 1,
        "False": 0,
        "true": 1,
        "false": 0
    })

    df.to_csv(FILE_PATH, index=False)
    print("'comments_disabled' sütunundaki True/False değerleri 1 ve 0 olarak dönüştürüldü")
else:
    print("'comments_disabled' sütunu bulunamadı.")


&nbsp;
## Veri'nin Toplanması
Kaggle'dan aldığımız dataset'te sadece trend olan videoların verisi vardı ve bizim modelimizin tahmin yapabilmesi için non trend video verilerine ihtiyacımız vardı. İşte burada Youtube Data API v3 devreye giriyor ve aşağıdaki kod yardımıyla bütün takım üyeleri veri çekmiştir bu API aracılığıyla. Tabi bunun çalışması için Google Cloud Console sitesinden API key oluşturulup veri çekimi için kod kısmına eklenmesi gerekir.

In [ ]:
API_KEYS = []

OUTPUT_FILE = "../Dataset/non_trending_videos.csv"
MAX_RESULTS = 50
TARGET_COUNT = 100000
SAVE_INTERVAL = 1000

KEYWORDS = [
    "music", "movie", "vlog", "review", "funny", "gaming", "education", "sports",
    "tutorial", "travel", "science", "art", "food", "comedy", "documentary",
    "tech", "dance", "live", "shorts", "how to", "challenge", "reaction",
    "asmr", "interview", "test", "study", "ai", "robotics", "fashion", "nature"
]

if os.path.exists(OUTPUT_FILE):
    df_existing = pd.read_csv(OUTPUT_FILE)
    existing_ids = set(df_existing["video_id"].astype(str))
    print(f"Existing records loaded: {len(existing_ids)} entries\n")
else:
    existing_ids = set()
    print("New CSV will be created\n")

all_videos = []
total_count = 0
api_index = 0
quota_exhausted = [False] * len(API_KEYS)
page_token = None

def get_key():
    global api_index
    return API_KEYS[api_index % len(API_KEYS)]


def next_key():
    global api_index
    quota_exhausted[api_index] = True
    usable = [i for i, used in enumerate(quota_exhausted) if not used]

    if not usable:
        print("All API keys have reached their quota. Stopping data collection...")
        save_progress()
        exit(0)

    api_index = usable[0]
    print(f"Switched to next API key → {get_key()}")

def save_progress():
    global all_videos
    if not all_videos:
        return
    print(f"Saving {len(all_videos)} videos...")
    temp_df = pd.DataFrame(all_videos)
    mode = "a" if os.path.exists(OUTPUT_FILE) else "w"
    header = not os.path.exists(OUTPUT_FILE)
    temp_df.to_csv(OUTPUT_FILE, mode=mode, index=False, header=header)
    all_videos.clear()
    print(f"{OUTPUT_FILE} updated.")

def fetch_video_stats(video_ids):
    stats = {}
    for i in range(0, len(video_ids), 50):
        batch = video_ids[i:i + 50]
        while True:
            url = (
                f"https://www.googleapis.com/youtube/v3/videos"
                f"?part=statistics,snippet&id={','.join(batch)}&key={get_key()}"
            )
            try:
                resp = requests.get(url, timeout=10)
                data = resp.json()

                if "error" in data:
                    msg = data["error"]["message"]
                    print(f"{msg}")
                    if "quota" in msg.lower():
                        next_key()
                        continue
                    else:
                        break

                for item in data.get("items", []):
                    vid = item["id"]
                    s = item.get("statistics", {})
                    snippet = item.get("snippet", {})
                    stats[vid] = {
                        "view_count": int(s.get("viewCount", 0)),
                        "likes": int(s.get("likeCount", 0)),
                        "comment_count": int(s.get("commentCount", 0)),
                        "categoryId": int(snippet.get("categoryId", 0)),
                        "tags": ", ".join(snippet.get("tags", [])),
                        "comments_disabled": 1 if "commentCount" not in s else 0,
                        "ratings_disabled": 1 if "likeCount" not in s else 0
                    }

                break
            except Exception as e:
                print("Error fetching video statistics:", e)
                next_key()
                time.sleep(1)
        time.sleep(0.15)
    return stats

def random_date_range_2022_2024():
    start = datetime(2022, 1, 1)
    end = datetime(2024, 12, 31)
    delta = end - start
    random_start = start + timedelta(days=random.randint(0, delta.days - 30))
    random_end = random_start + timedelta(days=30)
    return (
        random_start.strftime("%Y-%m-%dT00:00:00Z"),
        random_end.strftime("%Y-%m-%dT23:59:59Z"),
    )


print("Starting video collection (US region, 2022–2024 range)...\n")

for _ in tqdm(range(TARGET_COUNT // MAX_RESULTS * 2), desc="API requests"):
    query = random.choice(KEYWORDS)
    published_after, published_before = random_date_range_2022_2024()

    search_url = (
        f"https://www.googleapis.com/youtube/v3/search"
        f"?part=snippet&type=video&maxResults={MAX_RESULTS}"
        f"&regionCode=US&q={query}&videoDuration=any"
        f"&publishedAfter={published_after}&publishedBefore={published_before}"
        f"&key={get_key()}"
    )

    if page_token:
        search_url += f"&pageToken={page_token}"

    try:
        response = requests.get(search_url, timeout=10)
        data = response.json()
    except Exception as e:
        print("API request error:", e)
        next_key()
        continue

    if "error" in data:
        msg = data["error"]["message"]
        print(f"{msg}")
        if "quota" in msg.lower():
            next_key()
        continue

    page_token = data.get("nextPageToken", None)
    items = data.get("items", [])
    new_ids = []

    for item in items:
        if item["id"].get("kind") != "youtube#video":
            continue

        vid_id = item["id"].get("videoId")
        if not vid_id or vid_id in existing_ids:
            continue

        snippet = item["snippet"]
        all_videos.append({
            "video_id": vid_id,
            "title": snippet.get("title", ""),
            "publishedAt": snippet.get("publishedAt", ""),
            "channelId": snippet.get("channelId", ""),
            "channelTitle": snippet.get("channelTitle", ""),
            "description": snippet.get("description", ""),
            "thumbnail_link": snippet.get("thumbnails", {}).get("default", {}).get("url", ""),
            "is_trending": 0
        })
        existing_ids.add(vid_id)
        new_ids.append(vid_id)
        total_count += 1

    if new_ids:
        stats_data = fetch_video_stats(new_ids)
        for v in all_videos[-len(new_ids):]:
            vid = v["video_id"]
            if vid in stats_data:
                v.update(stats_data[vid])

    if total_count % SAVE_INTERVAL == 0 and total_count > 0:
        save_progress()
        print(f"Total videos collected: {total_count}\n")

    if total_count >= TARGET_COUNT:
        print("Target reached!")
        break

    time.sleep(0.25)

save_progress()
print(f"\nProcess completed. Total videos collected: {total_count}")
print(f"CSV file: {os.path.abspath(OUTPUT_FILE)}")


Çekilen verilerin `publishedAt` feature'ında bulunan tarihler ile kaggle'dan alınan datasetin `publishedAt` feature'ının tarih formatıyla uyuşmuyordu. Bu yüzden O sütundaki tarihleri birbirleriyle denkleştirmek için aşağıdaki kod bloğundan yararlanılmıştır.

In [ ]:
df["publishedAt"] = pd.to_datetime(df["publishedAt"], errors="coerce", utc=True)
df["publishedAt"] = df["publishedAt"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")
df.to_csv(FILE_PATH, index=False)

print("Tarih formatları başarıyla ISO-Z formatına dönüştürüldü.")


Bu işlem sonrasında orijinal(Kaggle'dan alınan) ve bizim elimizde olan datasetlere *`is_trending`* feature'ı ekleyip **Trend** verisi içeren kaggle dataset'i ile bizim çekmiş olduğumuz **Non Trend** verisi içeren datasetleri merge ederek şuan elimizde olan ve kullanmış olduğumuz US'te paylaşılan videoların *`Final_Dataset`*'ini elde etmiş oluyoruz.